In [2]:
""" 
Run nSimplices + MDS vs MDS on subset (STOOL and VAGINA only) of HMP dataset 
"""

' \nRun nSimplices + MDS vs MDS on subset (STOOL and VAGINA only) of HMP dataset \n'

In [2]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import pandas as pd
import random as alea
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold import MDS


data_dir = "../data/"
output_dir = "../outputs"
target_sites = ['STOOL', 'VAGINA']
target_colors = ["cornflowerblue", "orange"]

In [4]:
"""
Prepare dataset
"""

# iterate over QE and NB
files = ["hmp_v13lqphylotypeQuantNB_rs_c.csv", "hmp_v13lqphylotypeQuantE_rs_c.csv"]
color_df = pd.read_csv(os.path.join(data_dir, "hmp_v13lqphylotypePheno_rs_c.csv"), header=0)

for file in files:
    data_path = os.path.join(data_dir, file)
    df_hmp_ori = np.loadtxt(data_path, delimiter=",")
    df_hmp = []


    for index, row in color_df.iterrows():
        site_exist = False
        for site in target_sites:
            if row[site]:
                site_exist = True
        if site_exist:
            df_hmp.append(df_hmp_ori[index])
    df_hmp = np.array(df_hmp)
    np.savetxt(os.path.join(data_dir, file[:-8]+"target_"+file[-8:]), df_hmp, fmt='%1.17f', delimiter=',')


In [5]:
""" 
Prepare colors
"""

colors = []
new_color_df = color_df.copy(deep = True)
drop_indices = []

for index, row in color_df.iterrows():
    site_exist = False
    for i in range(len(target_sites)):
        site = target_sites[i]
        if row[site]:
            colors.append(target_colors[i])
            site_exist = True
    if not site_exist:
        drop_indices.append(index)

colors = np.array(colors)
print(colors.shape)

np.savetxt(os.path.join(data_dir, "hmp_target_colors.txt"), colors, fmt="%s")

new_color_df = new_color_df.drop(drop_indices)
new_color_df.to_csv(os.path.join(data_dir, "hmp_v13lqphylotypePheno_target_rs_c.csv"), header=True, index=False)

(330,)


In [6]:
""" 
Run nSimplices on HMP dataset
"""
colors = np.loadtxt(os.path.join(data_dir, "hmp_target_colors.txt"), dtype="str")
exec(open("../nsimplices.py").read())
alea.seed(42)


In [29]:
""" 
Run
(1) NB normalization + nSimplices + cMDS 
(2) QE normalization + nSimplices + cMDS 

To derive the axes data
""" 

output_files = ["hmp_NB_nSimplices_target_cMDS_axes.txt", "hmp_QE_nSimplices_target_cMDS_axes.txt"]
data_files = ["hmp_v13lqphylotypeQuantNB_target_rs_c.csv", "hmp_v13lqphylotypeQuantE_target_rs_c.csv"]
for i in range(len(output_files)):
    output_file = output_files[i]
    data_file = data_files[i]
    axes_output_path = os.path.join(output_dir, output_file)
    print("======== NB/QE normalization + nSimplices + cMDS ========")
    data_path = os.path.join(data_dir, data_file)
    df_hmp = np.loadtxt(data_path, delimiter=",")
    hmp_dis_sq=squareform(pdist(df_hmp))

    feature_num = 11
    dim_start = 11
    dim_end = feature_num

    print("hmp_dis_sq shape is:", hmp_dis_sq.shape)
    outlier_indices, subspace_dim , corr_pairwise_dis, corr_coord = nsimplices(hmp_dis_sq, feature_num, dim_start, dim_end)
    print("subspace dimension is:", subspace_dim)

    # run cMDS to get the corrected coordinates in importance decreasing order
    _, _, Xe = cMDS(corr_pairwise_dis)
    np.savetxt(axes_output_path, Xe, fmt='%f')

======== NB/QE normalization + nSimplices + cMDS ========
hmp_dis_sq shape is: (330, 330)
dim in find_subspace_dim is: 11
med_height is: [1.59165242]
subspace_dim one is: 11
thres is: 2.2981567268056518 mean is: 1.6544958073728433 std is: 0.21455363981093611
outlier indices are: [  0  15  20  44 152 199 267 275 277 279 288 304]
idx is: 0 height is: 2.632114582197141 thres is: 2.2981567268056518
idx is: 15 height is: 2.3380998398961506 thres is: 2.2981567268056518
idx is: 20 height is: 2.3468527813716715 thres is: 2.2981567268056518
idx is: 44 height is: 2.62774727576527 thres is: 2.2981567268056518
idx is: 152 height is: 2.523850581410362 thres is: 2.2981567268056518
idx is: 199 height is: 2.8034838753740647 thres is: 2.2981567268056518
idx is: 267 height is: 2.588977835882888 thres is: 2.2981567268056518
idx is: 275 height is: 2.4153786814302167 thres is: 2.2981567268056518
idx is: 277 height is: 2.313291291727865 thres is: 2.2981567268056518
idx is: 279 height is: 2.453776719126483 t

/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


outliet_indices is: [  0  15  20  44 152 199 267 275 277 279 288 304]
original coord is: [ 0.06694534 -0.81576918 -0.93970248  0.26859415  1.36426167 -0.92988582
 -1.14322428  0.13417262  0.24816972 -1.54522231 -0.38538546]
proj_coord is: [-0.10771837 -0.36917386  0.05782183 -0.12406843  0.1929839   0.3871508
 -0.41748364  0.23119821 -0.03820774 -0.00194076 -0.44870754]
proj_coord is: [ 0.08503506 -0.19848618  0.56031094  0.00902267  0.5129978   0.2675502
 -0.59301994 -0.11476834  0.32497537 -0.41782265 -0.61708582]
proj_coord is: [-0.31755102 -0.0933298   0.27851617 -0.07595684  0.41126606 -0.18901897
 -0.67148204  0.03325966  0.76448525 -0.44916081 -0.94547523]
proj_coord is: [-0.41590294 -0.46834896  0.23550506  0.1580606   0.47019416 -0.38828945
 -0.48753566 -0.25757212  0.57770434 -0.44506152 -1.13526699]
proj_coord is: [-0.53987748 -0.59715391  0.24956606  0.29805419  0.53313686 -0.32595472
 -0.49012642 -0.20225353  0.73825133 -0.44754915 -0.93831973]
proj_coord is: [-0.50406536 

/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


outliet_indices is: [199 267]
original coord is: [-0.98952832  0.0204523  -0.32529787 -1.66859173  1.05182242 -0.97532354
 -0.54712718 -0.46688831  1.00433233 -0.51088199 -0.224979  ]
proj_coord is: [ 0.00215576 -0.02475141 -0.00067212  0.02307497  0.00921571 -0.04507611
  0.00850637  0.0270181   0.01153527 -0.00068232  0.01340727]
proj_coord is: [-0.09454753  0.1151049   0.14208186 -0.03800158 -0.1198721  -0.17583491
 -0.1133209   0.1188128  -0.01884288  0.0620001  -0.02982937]
proj_coord is: [-0.05299653 -0.3685145   0.05965203 -0.77889253 -0.10589908 -0.27751992
 -0.0441838   0.3633731  -0.28376623 -0.06828081 -0.32520124]
proj_coord is: [-0.20347915 -0.38738632  0.06129112 -0.71127708 -0.20905611 -0.31796464
  0.02350141  0.28619537 -0.27604431 -0.3126184  -0.42369995]
proj_coord is: [-0.30510318 -0.29803079  0.12014539 -0.75429747 -0.10928001 -0.23516531
  0.17968987  0.43282432 -0.21699864 -0.33416574 -0.40209051]
proj_coord is: [-0.14387354 -0.35618737  0.39142064 -0.88334107 -0

In [44]:
""" 
Run
(1) NB normalization + cMDS 
(2) QE normalization + cMDS 

To derive the axes data
""" 

axes_files = ["hmp_NB_target_MDS_cMDS_axes.txt", "hmp_QE_target_MDS_cMDS_axes.txt"]
data_files = ["hmp_v13lqphylotypeQuantNB_target_rs_c.csv", "hmp_v13lqphylotypeQuantE_target_rs_c.csv"]

for i in range(len(axes_files)):
    axes_file = axes_files[i]
    data_file = data_files[i]

    print("======== QE/NB normalization + MDS + cMDS ========")
    data_path = os.path.join(data_dir, data_file)
    axes_output_path = os.path.join(output_dir, axes_file)

    df_hmp = np.loadtxt(data_path, delimiter=",")
    hmp_dis_sq=squareform(pdist(df_hmp))

    # Plot cMDS embedding using the pairs of axis from the four most significant axes 
    enforce_dim = 11 # enforcing dimension 11 to be consistent with QE+nsimplices+cMDS
    embedding = MDS(n_components=enforce_dim) 
    corr_coord = embedding.fit_transform(hmp_dis_sq)
    corr_dis_sq=squareform(pdist(corr_coord))
    _, _, Xe = cMDS(corr_dis_sq)

    np.savetxt(axes_output_path, Xe, fmt='%f')

======== QE/NB normalization + MDS + cMDS ========


/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:601: UserWarning: The MDS API has changed. ``fit`` now constructs an dissimilarity matrix from data. To use a custom dissimilarity matrix, set ``dissimilarity='precomputed'``.
  warnings.warn(
/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


======== QE/NB normalization + MDS + cMDS ========


/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:601: UserWarning: The MDS API has changed. ``fit`` now constructs an dissimilarity matrix from data. To use a custom dissimilarity matrix, set ``dissimilarity='precomputed'``.
  warnings.warn(
/home/wanxinli/.local/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


In [22]:
""" 
Plot pairwise result
"""

figure_files = ["hmp_target_NB_nSimplices_cMDS.png", "hmp_target_QE_nSimplices_cMDS.png", "hmp_target_NB_MDS_cMDS.png", "hmp_target_QE_MDS_cMDS.png"]
axes_files = ["hmp_target_NB_nSimplices_cMDS_axes.txt", "hmp_target_QE_nSimplices_cMDS_axes.txt", "hmp_target_NB_MDS_cMDS_axes.txt", "hmp_target_QE_MDS_cMDS_axes.txt"]
titles = ["NB+nSimplices", "QuantE+nSimplices", "NB+MDS", "QuantE+MDS"]
num_axes = 3 # show pairwise 2D plot to decompose the 3D plot

for i in range(len(figure_files)):
    figure_file = figure_files[i]
    axes_file = axes_files[i]
    title = titles[i]

    print("======== plot pairwise 2D plot (subset) ========")
    Xe = np.loadtxt(os.path.join(output_dir, axes_file))
    print(Xe.shape)
    for first_dim in range(num_axes):
        for second_dim in range(first_dim+1, num_axes):
            plt.figure()
            # only plot stool (blue), ears (black), throat (pink) points 

            stool_indices = [i for i, e in enumerate(colors) if e == 'cornflowerblue']
            ears_indices = [i for i, e in enumerate(colors) if e == 'orange']

            plt.scatter(Xe[stool_indices, second_dim], \
                Xe[stool_indices, first_dim], s=5, c='cornflowerblue', label = "STOOL")
            plt.scatter(Xe[ears_indices, second_dim], \
                Xe[ears_indices, first_dim], s=5, c='orange', label = "VAGINA")


            # for i in range(Xe.shape[0]):
            #     plt.scatter(Xe[i, second_dim], Xe[i, first_dim], s=5, c=colors[i])
            plt.legend()
            plt.title(title, size=10)   
            plt.savefig(os.path.join(output_dir, figure_file[:-4]+"_"+str(first_dim)+"_"+str(second_dim)+".png"))
            

======== plot pairwise 2D plot (subset) ========
(330, 163)
======== plot pairwise 2D plot (subset) ========
(330, 165)
======== plot pairwise 2D plot (subset) ========
(330, 165)
======== plot pairwise 2D plot (subset) ========
(330, 161)
